# Analyzing your Runs

In this tutorial you will learn: 

1. How to visualize and analyze your runs

In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
from bokeh.palettes import Colorblind
output_notebook()
import numpy as np
import pandas as pd
import astropy.units as u

#here is pyeddy
import pyeddy as ed
import justplotit as jpi


Loading BokehJS ...

In [2]:
pts = pd.read_csv(jdi.jupiter_pt(),delim_whitespace=True)
pts1 = pd.read_csv('/Users/nbatalh1/Downloads/hat1.pt',delim_whitespace=True, usecols=[1,2,3],
                  names = ['pressure','temperature','kz'],skiprows=1)


Let's just start with the simple isothermal profile example that we did before

In [3]:
pressure = pts['pressure'].values#np.logspace(-5,2,30) #simple isotherml PT profile (kelvin)
temperature=pts['temperature'].values #= np.zeros(30)+1300  
kz = np.zeros(61)+ 1e8 #constant kz profile

metallicity = 1 #atmospheric metallicity relative to Solar
mean_molecular_weight = 2.2 # atmospheric mean molecular weight

#set the run 
a = ed.Atmosphere(['MnS','Cr','MgSiO3','Al2O3','Fe'],#,'ZnS''Na2S'
                  fsed=0.3,mh=metallicity,
                 mmw = mean_molecular_weight)

#set the planet gravity
a.get_gravity(gravity=7.460, gravity_unit=u.Unit('m/(s**2)'))

#PT 
a.get_pt(df = pts1)#pd.DataFrame({'pressure':pressure, 'temperature':temperature,
#                           'kz':kz}))

#get full dictionary output 
all_out = ed.justdoit(a, as_dict=True,
                      directory='/Users/nbatalh1/Documents/codes/TheOGTC/input/optics/')

Virtual Cloud Found: MgSiO3
Virtual Cloud Found: Al2O3
Virtual Cloud Found: Fe


In [4]:
all_out.keys()

dict_keys(['pressure', 'pressure_unit', 'temperature', 'temperature_unit', 'wave', 'wave_unit', 'condensate_mmr', 'cond_plus_gas_mmr', 'mean_particle_r', 'droplet_eff_r', 'r_units', 'column_density', 'column_density_unit', 'opd_per_layer', 'single_scattering', 'asymmetry', 'opd_by_gas', 'condensibles', 'scalar_inputs'])

## PT Profile with Condensation Curves

Thick lines in this pt plot signify those gases that were turned on in the run, NOT those that were recommended. 

In [5]:
show(jpi.pt(all_out,plot_height=450))

/Users/nbatalh1/Documents/codes/pyeddy/root_functions.py:223: RuntimeWarning: divide by zero encountered in log10
  return np.log10(pv) - np.log10(partial_p)


## Integrated Optical Depth By Gas 

In [6]:
show(jpi.opd_by_gas(all_out))

## Use `PICASO` to visuzalize output

`PICASO` has a number of functions to visualize cloud output. We can use some eddysed functions to get the output into the specific format. 

In [7]:
#get into DataFrame format
df = ed.picaso_format(all_out['opd_per_layer'],
                      all_out['asymmetry'],all_out['single_scattering'])


In [22]:
import picaso.justplotit as pic_jpi

show(pic_jpi.plot_cld_input(nwno=len(all_out['wave']), nlayer=len(all_out['pressure']),
                        df=df,pressure=all_out['pressure']))


In [10]:
show(pic_jpi.plot_cld_input(196, 52, filename='/Users/nbatalh1/Downloads/inputTASH.cld',delim_whitespace=True))


In [11]:
fort = pd.read_csv('/Users/nbatalh1/Downloads/eddysed.out',delim_whitespace=True)
fort.keys()

Index(['z(m)', 'P(bar)', 'T(K)', 'qt_MnS', 'qt_Fe', 'qt_Na2S', 'qt_Al2O3',
       'qt_ZnS', 'qt_Cr', 'qt_MgSiO3', 'qc(g/g)_MnS', 'qc(g/g)_Fe',
       'qc(g/g)_Na2S', 'qc(g/g)_Al2O3', 'qc(g/g)_ZnS', 'qc(g/g)_Cr',
       'qc(g/g)_MgSiO3', 'reff(um)_MnS', 'reff(um)_Fe', 'reff(um)_Na2S',
       'reff(um)_Al2O3', 'reff(um)_ZnS', 'reff(um)_Cr', 'reff(um)_MgSiO3',
       'rg_MnS', 'rg_Fe', 'rg_Na2S', 'rg_Al2O3', 'rg_ZnS', 'rg_Cr',
       'rg_MgSiO3', 'opd_gas_MnS', 'opd_gas_Fe', 'opd_gas_Na2S',
       'opd_gas_Al2O3', 'opd_gas_ZnS', 'opd_gas_Cr', 'opd_gas_MgSiO3'],
      dtype='object')

In [12]:
fig = figure(x_axis_type='log',y_axis_type='log',y_range=[1e2,1e-6],height=400,
            title='OPD By Gas')
from bokeh.palettes import magma
cols = Colorblind[7]#magma(len(all_out['condensibles']))
ii=0
for i,c in zip(all_out['condensibles'],cols):
    fig.line(fort['opd_gas_'+i], fort['P(bar)'],color=c,line_width=4,legend_label=i)
    fig.line(all_out['opd_by_gas'][:,ii],all_out['pressure'],color=c,line_width=4,line_dash='dashed' )
    ii+=1
show(fig)

In [13]:
fig = figure(x_axis_type='log',y_axis_type='log',y_range=[1e2,1e-6],height=200)
from bokeh.palettes import magma
cols = Colorblind[7]
ii=0
for i,c in zip(all_out['condensibles'],cols):
    fig.line(fort['rg_'+i]*1e4, 
             fort['P(bar)'],
             color=c,
             line_width=4)
    
    fig.line(all_out['mean_particle_r'][:,ii],
             all_out['pressure'],
             color=c,
             line_width=4,
             line_dash='dashed' )
    ii+=1
show(fig)

In [14]:
fig = figure(x_axis_type='log',y_axis_type='log',y_range=[1e2,1e-6],height=200)
from bokeh.palettes import magma
cols = Colorblind[7]
ii=0
for i,c in zip(all_out['condensibles'],cols):
    fig.line(fort['qc(g/g)_'+i], 
             fort['P(bar)'],
             color=c,
             line_width=4)
    
    fig.line(all_out['condensate_mmr'][:,ii],
             all_out['pressure'],
             color=c,
             line_width=4,
             line_dash='dashed' )
    ii+=1
show(fig)

In [15]:
fig = figure(x_axis_type='log',y_axis_type='log',y_range=[1e2,1e-6],height=200)
from bokeh.palettes import magma
cols =Colorblind[7]
ii=0
for i,c in zip(all_out['condensibles'],cols):
    fig.line(fort['qt_'+i], 
             fort['P(bar)'],
             color=c,
             line_width=4)

    fig.line(all_out['cond_plus_gas_mmr'][:,ii],
             all_out['pressure'],
             color=c,
             line_width=4,
             line_dash='dashed' )
    ii+=1
show(fig)

In [21]:
df = pd.read_csv('/Users/nbatalh1/Downloads/inputTASH.cld',delim_whitespace=True)
opd_old = np.reshape(df['opd'].values,(52,196))
g_old =np.reshape(df['g0'].values,(52,196))
w_old =np.reshape(df['w0'].values,(52,196))
p_old =np.reshape(df['p'].values,(52,196))
fig = figure(y_axis_type='log',y_range=[1e2,1e-6],
             x_range=[0,1],height=200)
from bokeh.palettes import magma
cols =Colorblind[7]
ii=0
for i,c in zip(np.linspace(0,195,7),cols):
    fig.line(all_out['single_scattering'][:,int(i)], 
             all_out['pressure'],
             color=c,
             line_width=4)

    fig.line(w_old[:,int(i)], 
             p_old[:,int(i)]*1e-6,
             color=c,
             line_width=4,
            line_dash='dashed')
    ii+=1
show(fig)